In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

#Rock = 0, Paper = 1, Scissors = 2

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q -U kaggle_environments
import os #for file management

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt #plotting import
import seaborn as sns #basic stat stuff

from kaggle_environments import make, evaluate, utils #kaggle environments

import torch

In [ ]:
matrix = np.ones((3,3,3))
print(matrix[2,2,:])
print(matrix[0,0,:].argmax())

Create a random agent for testing

In [ ]:
%%writefile random.py
import numpy as np
def random_agent(observation, configuration):
    return int(np.random.randint(3))

In [ ]:
%%writefile submission.py
import numpy as np
import json
#import torch

matrix = np.ones((3,3,3)) * (1/3) #so we can choose object based on what we chose and what the opponent chose transition matrix
matrix_freq = np.ones((3,3,3)) #frequency matrix
prev_me = 0
prev_op = 0
#print(state_dict)

def copy_opponent_agent (observation, configuration):
    
    global prev_me, prev_op, matrix, matrix_freq
        
    if observation.step > 0:
        #return (observation.lastOpponentAction + 1)%3
        #prev_op = observation.lastOpponentAction #we store the last action of the opponent
        
        #from step > 1 we can update matrix because we know what we chose and what it chose
        if observation.step > 1:
            matrix_freq[prev_op, prev_me, observation.lastOpponentAction] += 1
            matrix[prev_op, prev_me, :] = matrix_freq[prev_op, prev_me, :] / np.sum(matrix_freq[prev_op, prev_me, :]) 
            
        
        prev_op = observation.lastOpponentAction #we store the last action of the opponent  
        
        #choose the optimal choice based on the transition matrix
        #choosing stochastically
        prev_me = (np.random.choice(3, p=matrix[prev_op, prev_me, :]) + 1) % 3
        
        #print(matrix) 
        
        state_dict = {"transition tensor" : matrix.tolist()}
        with open('transition_matrix.json', 'a') as outfile:
            json.dump(state_dict, outfile)
            outfile.write("\n")
        
        return prev_me
        
              
    else:
        #prev_me = np.random.randint(0,3)
        state_dict = {"transition tensor" : matrix.tolist()}
        with open('transition_matrix.json', 'w') as outfile:
            json.dump(state_dict, outfile)
            outfile.write("\n")
        prev_me = (np.random.choice(3, p=matrix[prev_op, prev_me, :]) + 1)%3
        return prev_me
        #json.dump(matrix_freq, outfile)

Running the environment

In [ ]:
env = make("rps", debug = True) #create the simulation environment

env.run(["submission.py", "statistical"]) #run it against random opponent
env.render(mode="ipython", width=500, height=450) #render it

In [ ]:
#import json
#with open('transition_matrix.json') as json_file:
#    bandit_state = json.load(json_file)
#print(bandit_state)

In [ ]:
env.reset()
# Play as the first agent against default "random" agent.
env.run(["submission.py", "reactionary"])
env.render(mode="ipython", width=500, height=450)

In [ ]:
env.reset()
# Play as the first agent against default "random" agent.
env.run(["submission.py", "counter_reactionary"])
env.render(mode="ipython", width=500, height=450)

In [ ]:
env.reset()
# Play as the first agent against default "random" agent.
env.run(["submission.py", "random.py"])
env.render(mode="ipython", width=500, height=450)